# Adiabatic Flame for  1D flow of n-decane model from RMG using ctwrap
This model from RMG is the detailed model of n-decane and we calculate the flame speed for a 1D reactor using the (ctwrap.modules.adiabatic_flame) from cantera and ctwrap

## 1. Function/Class Definitions

In [1]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [2]:
import sys
sys.path.append('/scratch/ajulu.c/anaconda3/envs/flame_env/')

In [3]:
!which python
!conda list flame_env

/scratch/ajulu.c/anaconda3/envs/flame_env/bin/python
# packages in environment at /scratch/ajulu.c/anaconda3/envs/flame_env:
#
# Name                    Version                   Build  Channel


In [4]:
import cantera as ct
import ctwrap as cw

## 2. Simulations

### 2.1 Create Simulation Object

In [5]:
sim = cw.Simulation.from_module(cw.modules.adiabatic_flame)
sim

In [6]:
%%time 
sim.run()

    defaults: mixture-averaged flamespeed = 0.711265 m/s
    defaults: multi-component flamespeed  = 0.727009 m/s
CPU times: user 808 ms, sys: 43.6 ms, total: 851 ms
Wall time: 846 ms


### 2.2 Create Simulation Handler

In [7]:
s = cw.SimulationHandler.from_yaml('adiabatic_flame_nc10.yaml', verbosity=1)
s

Simulations for entry `upstream.phi` with values: [0.5, 0.65, 0.75, 0.85, 0.95, 1.0, 1.25, 1.35, 1.45, 1.5, 1.55, 1.65, 1.75, 1.85, 1.95, 2.0]


### 2.3 Using Cantera to check the gas solution and the number of reaction and species

In [9]:
gas = ct.Solution('/scratch/ajulu.c/Code/GTL/nc10/chemkin/reduced_283.cti')



**** WARNING ****
For species HOCHO(370), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value computed using high-temperature polynomial: -38.1678


In [10]:
gas()


  reduced_283:

       temperature             300  K
          pressure          101325  Pa
           density         1.13796  kg/m^3
  mean mol. weight         28.0135  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          2076.5        5.817e+04     J
   internal energy          -86964       -2.436e+06     J
           entropy            6843        1.917e+05     J/K
    Gibbs function     -2.0508e+06       -5.745e+07     J
 heat capacity c_p          1038.3        2.909e+04     J/K
 heat capacity c_v          741.48        2.077e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                N2              1                1         -23.0324
     [ +282 minor]              0                0



In [11]:
num_species = gas.species()
num_rxns = gas.reactions()

if len(num_species) & len(num_rxns) > 0:
    print ("The number of species is {} and The number of reactions is {}".format(len(num_species), len(num_rxns)))
else: 
    print ("empty")

The number of species is 283 and The number of reactions is 8753


### 2.4 Run Simulation Tasks in Parallel

In [ ]:
%%time
s.run_parallel(sim, verbosity=0)



**** WARNING ****
For species HOCHO(370), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value computed using high-temperature polynomial: -38.1678


**** WARNING ****
For species HOCHO(370), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value computed using high-temperature polynomial: -38.1678


**** WARNING ****
For species HOCHO(370), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value computed using high-temperature polynomial: -38.1678


**** WARNING ****
For species HOCHO(370), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value computed using high-temperature polynomial: -38.1678


**** WARNING ****
For species HOCHO(370), discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  -40.3604
	Value com

## 3. Create Graphical Output

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

col = mpl.rcParams['axes.prop_cycle'].by_key()['color']
n_col = len(col)

### 3.1 Load Data
Loading data from output file

In [ ]:
data = cw.fileio.from_hdf(s.output_name)


In [ ]:
mix = {k.split('<')[0]: data[k] for k in data if 'mix' in k}
mlt = {k.split('<')[0]: data[k] for k in data if 'multi' in k}

tasks = data.get('variation')['tasks']
phi = data.get('variation')['values']

In [ ]:
up = data.get('defaults')['upstream']

### 3.2 Plot
plot flame speeds

In [ ]:
# create figure
fig, ax = plt.subplots(1)

# plot results

u_mix = [ mix[t]['u (m/s)'][0] for t in tasks]
u_mlt = [ mlt[t]['u (m/s)'][0] for t in tasks]

ax.plot(phi, u_mix, marker='o',
       linestyle='none', label='mixture-averaged')

ax.plot(phi, u_mlt, marker='s',
       markerfacecolor='none', linestyle='none', label='multi-component')


# add title/axis labels

ax.set_xlabel('Equivalence ratio (-)')
ax.set_ylabel('Flame speed (m/s)')

ax.set_title('Adiabatic premixed flame simulation for {} with {}'.format(up['fuel'],up['oxidizer']))


# add legend

leg = ax.legend(loc='lower right', title='Transport Model')

_ = leg.draw_frame(False)